In [18]:
import pandas as pd, openpyxl as op

# Convert Excel-style range (e.g., 'A1:B2') to a pair of index tuples (start_row, start_col, end_row, end_col)
def excel_range_to_indices(range_str):
    start_cell, end_cell = range_str.split(':')
    start_row, start_col = op.utils.cell.coordinate_to_tuple(start_cell)
    end_row, end_col = op.utils.cell.coordinate_to_tuple(end_cell)
    return start_row-1, start_col-1, end_row-1, end_col-1

def returnDataFromExcel(excel_range):
    excel_file = pd.ExcelFile("./Lab Data/sn excel sheet.xlsx")
    df = pd.read_excel(excel_file, sheet_name='Main')

    start_row, start_col, end_row, end_col = excel_range_to_indices(excel_range)

    subset_df = df.iloc[start_row:end_row+1, start_col:end_col+1]
    data_2d_array = subset_df.values.tolist()
    return data_2d_array

returnDataFromExcel(excel_range = 'C60:H73')


[[6.0, 6000, 3500, 0, 0, 0.05],
 [7.0, 6000, 3500, 0, 0, 0.05],
 [8.0, 6000, 3500, 0, 0, 0.02],
 [9.0, 6000, 3500, 0, 0, 0.03],
 [10.0, 6000, 3500, 0, 0, 0.03],
 [11.0, 6000, 3500, 0, 0, 0.02],
 [12.0, 6000, 3500, 0, 0, 0.025],
 [13.0, 6000, 3500, 0, 0, 0.025],
 [14.0, 6000, 3500, 0, 0, 0.05],
 [15.0, 6000, 3500, 0, 0, 0.075],
 [16.0, 6000, 3500, 0, 0, 0.075],
 [17.0, 6000, 3500, 0, 0, 0.1],
 [18.0, 6000, 3500, 0, 0, 0.03],
 [19.0, 6000, 3500, 0, 0, 0.03]]

In [19]:
import numpy as np, matplotlib.pyplot as plt


labbookdata = np.array(returnDataFromExcel("C60:H73"))




def spectroParamsExtract(filepath):
    
    with open(filepath, 'r') as file:
        file_contents = file.read()
        
    file_contents = file_contents.split("\n") 
    file_contents = file_contents[0] 
    file_contents = file_contents.split() 

    spectrometerData = np.array([float(num) for num in file_contents]) 
    
    return spectrometerData # exporting the number array


filearray = labbookdata[:,0] # file indices of interest here
filearray = [(int)(i) for i in filearray]
filearray

[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [20]:


from matplotlib.backends.backend_pdf import PdfPages
with PdfPages("Day2_Xe_Ne_plots.pdf") as pdf:
    
    k = 0 
    
    for i in filearray: 
        spectroParamFile = f"./Lab Data/Data_240619/rdm24-06-19_{i}.txt"
        datafile = f'./Lab Data/Data_240619/24-06-19_{i}_ch2.txt'
        

        spectrometerParameters = spectroParamsExtract(spectroParamFile)
        
        data = np.loadtxt(datafile, delimiter=' ');
        timeDelay = data[:, 0]
        intensity = data[:, 1]
        
        vgradient = f"6000 3500 0 -1750"
        
        # plotting the data
        plt.plot(timeDelay,intensity, label=f"power={labbookdata[k][5]*100}%", linewidth = 0.2, color="orange")
        plt.xlabel("Time Delay (in micro-s)", fontsize=8)
        plt.ylabel("Signal Intensity - Detector (mV) ", fontsize=8)
        plt.legend()
        
        k+=1
        
        plt.title(f"Laser Iterations {spectrometerParameters[0]}; Voltage Gradient : "+vgradient, fontsize=8)
        plt.suptitle(f"File #{i}; Xe doped He droplets", fontsize=12)
        
        pdf.savefig()
        plt.close()
        